## Four types of trusses in different gravity fields. 

Red color represents compression and blue color represents tension. The intensity of the color reflects the magnitude of the normalized load in the block under consideration.

In [3]:
import pyglet, random, math, os, sys, numpy as np, copy
from pyglet.window import key, FPSDisplay
from pyglet import image, shapes

Ws = 850; Hs = 350  # width and hight screen  
Kspring = 100.; Cspring = .3  # Default elasticity coefficient and damping coefficient
COUNTER = 0                   # counter

# begin coordinat left down corner
window = pyglet.window.Window(Ws,Hs,"Ferms",resizable = False)
batch = pyglet.graphics.Batch()
fps_display = FPSDisplay(window)
fps_display.label = pyglet.text.Label(font_name='Arial', font_size=12, x=20, y=20, color = (0,0,0,200))
label = pyglet.text.Label('', font_name='Arial', bold = False, font_size=19, x=335, y=Hs-160, color = (0,0,0,200), batch=batch)
#label.text = 'a) 2 nodes x 1 spring               b) 3 nodes x 3 springs              c) 4 nodes x 4 springs'  
pyglet.gl.glClearColor(1,1,1,1)


TS = np.float64(.03)        # time scale
COUNTER = 0                 # counter
ON = False                  # running the model
GRAV = 0                    # gravitation

# =================== Granule cell - Parallel fabers - Phasal space ======================== 
# для выделения Node - звездочка
selNode = shapes.Star(-10, -10, 1, 1, 5, rotation=0, color=(255, 230, 230))
Nodes = np.array([]); Springs = np.array([])
#============= LOAD NODES и SPRINGS #=============
gcpfN = np.float64(np.loadtxt('nodes_ferms.gcpf'))
if len(gcpfN.shape) == 1: gcpfN = np.reshape(gcpfN, (-1, 9))  # protection for the case of a single element

gcpfS = np.float64(np.loadtxt('strings_ferms.gcpf'))
if len(gcpfS.shape) == 1: gcpfS = np.reshape(gcpfS, (-1, 9))  # protection for the case of a single element

for n in gcpfN:
    radius = n[0]** (1./3.) * 3  # radius depends on mass
    color = (50,50,50) if n[7] else (200, 100, 100)
    Nodes = np.append (Nodes, shapes.Circle(n[1],n[2], radius,  color=color, batch=batch))
for s in gcpfS:
    Springs = np.append (Springs, shapes.Line(gcpfN[round(s[3])][1],gcpfN[round(s[3])][2], gcpfN[round(s[4])][1],gcpfN[round(s[4])][2], color=(120, 120, 120), width=2, batch=batch))

if len(gcpfN) == 0:
    gcpfN = np.empty(shape=[0, 9]) # parameters NODES [0..9] <=> (m, x,y, dx,dy, ddx,ddy, fix, select)
                               # fix - fixing node (T/F), select - selected node (T/F)
if len(gcpfS) == 0:
    gcpfS = np.empty(shape=[0, 9]) # parameters SPRINGS [0..9] <=> 
                               # (l0 - free distance, k - coefficient of elasticity, c - coefficient of resistance,
                               # body1 and body2 are the numbers of the bodies in the arrays that the spring connects)
                               # exter, inter, visible, select - selected node (T/F), F

nN = len(gcpfN); nS = len(gcpfS)
# ======================= visualization of bodies =============================
Nodes   = [shapes.Circle(0,0, 30,  color=(200, 100, 100), batch=batch) for i in range(nN)];
Springs = [shapes.Line  (0,0, 0,0, color=(100, 100, 100), width=4, batch=batch) for i in range(nS)]

for i in range(nN):
    Nodes[i].radius = gcpfN[i][0]** (1./3.) * 3         # radius depends on mass
    Nodes[i].x = gcpfN[i][1]; Nodes[i].y = gcpfN[i][2]  # coordinate nodes
    
for i in range(nS):
    b1i = int(gcpfS[i,3]); b2i = int(gcpfS[i,4])  # indexes (numbers) of the first and second node
    Springs[i].x  = gcpfN[b1i,1]; Springs[i].y  = gcpfN[b1i,2]
    Springs[i].x2 = gcpfN[b2i,1]; Springs[i].y2 = gcpfN[b2i,2]

Lure = shapes.Circle(0,0, 15,  color=(100, 100, 200)) #, batch=batch)  

l1 = shapes.Line(35, 250, 795, 250, width=2, color=(244, 125, 2), batch=batch)
l2 = shapes.Line(35, 100, 795, 100, width=2, color=(244, 125, 2), batch=batch) 
# ======================= drawing procedure =============================
@window.event
def on_draw():
    window.clear()
    #fps_display.draw()
    batch.draw()
# ======================= mouse press ============================
@window.event
def on_mouse_press(x, y, button, modifiers):
    global ON, GRAV
    if button == pyglet.window.mouse.LEFT:
        GRAV += .5 if GRAV < 7 else -7
        
        ON = False if ON else True

@window.event
def on_mouse_motion(x, y, dx, dy):
    Lure.x = x
    Lure.y = y
# ======================= update procedure ============================
def update(dt):  
    global Springs,Nodes,gcpfN, gcpfS, ON, COUNTER, TS, batch, Lure, GRAV
    
    minSF, maxSF = gcpfS[:,9].min(), gcpfS[:,9].max() 
    norm = 155/(abs(minSF) if abs(minSF) > abs(maxSF) else 155/abs(maxSF)) 
    label.text = 'Gravitation - ' + str(GRAV)
    #'F min.max = ' + str(round(minSF,0)) + ', ' + str(round(maxSF,0))
    
    # conjugation of the phase space of bodies with the visual representation
    for i in range(nN):        
        Nodes[i].x = gcpfN[i][1]; Nodes[i].y = gcpfN[i][2]
    for i in range(nS):
        b1i = int(gcpfS[i,3]); b2i = int(gcpfS[i,4])  # indices (numbers) of the first and second bodies
        Springs[i].x  = gcpfN[b1i,1]; Springs[i].y  = gcpfN[b1i,2]
        Springs[i].x2 = gcpfN[b2i,1]; Springs[i].y2 = gcpfN[b2i,2]
        
        F = gcpfS[i,9]
        Springs[i].color = (F*norm+99,0,0) if F>0 else (0,0,-F*norm+99)
    

    Famax = 0; 
    for j in range(5):  # Improving the accuracy of the calculation by reducing the discreteness of the time period
        for i in range(nS):
            b1i = int(gcpfS[i,3]); b2i = int(gcpfS[i,4])  # indices (numbers) of the first and second bodies
        
            lt = gcpfS[i,0]    
    # ==================== time steam ========================
            lx = gcpfN[b1i,1]-gcpfN[b2i,1]          # lx = x1-x2
            ly = gcpfN[b1i,2]-gcpfN[b2i,2]          # ly = y1-y2
            lc = math.sqrt(lx*lx + ly*ly)           # l current
            F = gcpfS[i,1]*(lt-lc)                  # k*(lr-l)
            gcpfS[i,9] = F                          
            #Famax = Famax+1 if abs(F) > Famax else Famax-1
            Famax = abs(F) if abs(F) > Famax else Famax
            
         # acceleration calculation
            # 1 node
            ml1 = gcpfN[b1i,0]*lc                                             
            gcpfN[b1i,5] =  F*lx/ml1 - gcpfN[b1i,3]*gcpfS[i,2]/gcpfN[b1i,0]    # ddx1 = (F/m1) * (a/l) - dx*c/m1 
            gcpfN[b1i,6] =  -GRAV +F*ly/ml1 - gcpfN[b1i,4]*gcpfS[i,2]/gcpfN[b1i,0]    # ddy1 = (F/m1) * (b/l) - dy*c/m2
            # 2 node
            ml2 = gcpfN[b2i,0]*lc                                            
            gcpfN[b2i,5] = -F*lx/ml2 - gcpfN[b2i,3]*gcpfS[i,2]/gcpfN[b2i,0]    # ddx2 = -(F/m2) * (a/l) - dx*c/m1
            gcpfN[b2i,6] =  -GRAV -F*ly/ml2 - gcpfN[b2i,4]*gcpfS[i,2]/gcpfN[b2i,0]    # ddy2 = -(F/m2) * (b/l) - dy*c/m2
                                    
         # speed calculation
            # 1 node
            gcpfN[b1i,3] += gcpfN[b1i,5]*TS; gcpfN[b1i,4] += gcpfN[b1i,6]*TS    
            # 2 node 
            gcpfN[b2i,3] += gcpfN[b2i,5]*TS; gcpfN[b2i,4] += gcpfN[b2i,6]*TS    
         
         # new coordinates calculation
            if not gcpfN[b1i,7]: # 1 node
                gcpfN[b1i,1] += gcpfN[b1i,3]*TS; gcpfN[b1i,2] += gcpfN[b1i,4]*TS    
            if not gcpfN[b2i,7]: # 2 node
                gcpfN[b2i,1] += gcpfN[b2i,3]*TS; gcpfN[b2i,2] += gcpfN[b2i,4]*TS   
        
    COUNTER += 1  
# =========================== MAIN LOOP ============================
if __name__ == "__main__":
    pyglet.clock.schedule_interval(update, 1/50)
    pyglet.app.run()
    pyglet.clock.unschedule(update)   # necessary to restart the countdown of time intervals
#========================================================================